In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# データについて見てみる

## example_sample_submission.csv

In [ ]:
df = pd.read_csv("/kaggle/input/jane-street-market-prediction/example_sample_submission.csv")
df.head()

ファイルの中身は上記の通り。"ts_id"は時系列の順番でIDを割り当てている。"action”は1:トレードを実行、0:見送りを表している。この例では、すべて"action"は1となっている。kaggleに提出するときは、このデータ構造で提出する必要がある。

## example_test.csv

In [ ]:
df = pd.read_csv("/kaggle/input/jane-street-market-prediction/example_test.csv")
df.head()

ファイルの中身は上記の通り。テストデータ用のデータセットとなっている。"weight"は後述。"feature_X"は匿名の説明変数となっており、130個ある。"date"は取引日を表している。テストデータでは、"date"は0~2となっているので、３日間のデータということになる。モデルのテストはこのデータセットを用いて行い、この予測結果を提出する。モデルの学習では、使用してはいけない点に注意する。

## features.csv

In [ ]:
df = pd.read_csv("/kaggle/input/jane-street-market-prediction/features.csv")
df.head()

ファイルの中身は上記の通り。各行に匿名変数"feature_X"に対応しており、各列に匿名変数のメタ情報として”tag_X”が格納されている。メタ情報は、28個からなっており、値は2値(TRUE/FALSE)となっている。情報が非公開過ぎて何を表しているのかわからない…。28個のカテゴリがあって、各説明変数は、それぞれのカテゴリに属しているか否かといった感じだろうか。この辺のデータもうまく使わないと、kaggle上位には入れないのだろう。

## train.csv

In [ ]:
df = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
df.loc[df.weight==0].size / df.size

ファイルの中身は上記の通り。データ数は、2,390,490個。"date"は500まであることから500日分のデータと思われる。2,390,490  ÷ 500(日) ÷ 24(h)  ÷ 60(min)  ≒ 3.3個/分であることから、約20秒ごとのデータといったところか。"weight"と”resp”はそれぞれの取引のリターンを表している。resp_{1,2,3,4}は、異なる時間軸におけるリターンである。おそらく、これは取引終了の期間幅を表しており、例えば、取引開始後の｛1時間後, 6時間後, 12時間後, 24時間後}といった意味合いだと思われる。尚、テストデータにはresp_{1,2,3,4}は含まれていない。”weight”がゼロのものは、補完のために意図的にデータセットに加えたもので、スコア評価には組み込まれないと説明にあった。"weight"の意味については、把握しきれていないが、0と0以外について特徴を見ておく必要はありそう。